In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from xtquant import xtdata, xttrader, xtconstant
from xtquant.xttype import (StockAccount)

In [2]:
from xtquant import xtdata
import pandas as pd
from datetime import datetime

# 读取持仓股票文件
holding_stocks_df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Daily_Signal\\holding_stocks.csv')

# 从持仓股票列表中获取股票代码列表
stock_list = holding_stocks_df['Stock Code'].dropna().tolist()

# 获取日期并设置开始时间
# 假设我们使用持仓股票列表中的最新日期作为开始日期
if not holding_stocks_df.empty:
    latest_date = pd.to_datetime(holding_stocks_df['Last Buy Date'].min()).strftime('%Y%m%d')
    start_time = f"{latest_date}140000"  # 设置为最新买入日期的下午14:00开始
else:
    start_time = datetime.now().strftime('%Y%m%d093000')  # 如果没有持仓股票，使用当前日期

# 设置结束时间为当前日期
end_time = datetime.now().strftime('%Y%m%d%H%M%S')

period = '1m'

def on_progress(data):
    print("下载进度：", data)

xtdata.download_history_data2(stock_list=stock_list, period=period, start_time=start_time, end_time=end_time, callback=on_progress,incrementally=True)

***** xtdata连接成功 *****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: D:\国金QMT交易端模拟\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息



In [3]:
from xtquant import xtdata
import pandas as pd
from datetime import datetime

# 读取持仓股票文件
holding_stocks_df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Daily_Signal\\holding_stocks.csv')

# 从持仓股票列表中获取股票代码列表
stock_list = holding_stocks_df['Stock Code'].dropna().tolist()

# 初始化存储历史数据的字典
history_data = {}

# 设置结束时间为当前日期
end_time = datetime.now().strftime('%Y%m%d%H%M%S')

period = '1m'

# 对每个股票代码进行数据获取
for stock in stock_list:
    # 获取每只股票的最新买入日期
    stock_data = holding_stocks_df[holding_stocks_df['Stock Code'] == stock]
    if not stock_data.empty:
        latest_buy_date = pd.to_datetime(stock_data['Last Buy Date'].max()).strftime('%Y%m%d')
        start_time = f"{latest_buy_date}140000"  # 设置为最新买入日期的下午14:00开始
    else:
        start_time = datetime.now().strftime('%Y%m%d140000')  # 如果没有买入日期，使用当前日期

    data = xtdata.get_market_data_ex(
        field_list=[],  # 字段
        stock_list=[stock],  # 当前股票代码
        period=period,  # 数据周期
        start_time=start_time,  # 数据起始时间
        end_time=end_time,  # 数据结束时间
        dividend_type='none',  # 除权方式
        fill_data=True  # 是否填充数据
    )
    if data:
        history_data[stock] = data[stock]  # 假设返回的数据是字典形式，键为股票代码

# 检查是否有数据
if not history_data:
    print("No Buy Signal")
else:
    # 将字典转换为 DataFrame，并按股票代码进行分组
    df_list = [pd.DataFrame(v).assign(stock_code=k) for k, v in history_data.items()]
    if df_list:
        df = pd.concat(df_list)
        # 重置索引，并将原索引列重命名为 'Datetime'
        df = df.reset_index().rename(columns={'index': 'Datetime'})
        
        # 确保 'stock_code' 列位于 'time' 列之后
        # 假设 'time' 列已存在于 DataFrame 中
        if 'time' in df.columns:
            # 获取除 'stock_code' 外的所有列
            cols = df.columns.tolist()
            # 移除 'stock_code'
            cols.remove('stock_code')
            # 找到 'time' 列的索引
            time_index = cols.index('time')
            # 在 'time' 列之后插入 'stock_code'
            cols.insert(time_index + 1, 'stock_code')
            # 重新排列 DataFrame
            df = df[cols]
        # print(df)
    else:
        print("No Buy Signal")
df

下载进度： {'finished': 1, 'total': 8, 'stockcode': '', 'message': '600938.SH'}


,Datetime,time,stock_code,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag


In [4]:
# 函数来实现 MACD 指标
def getmacd(data,short=24,long=39, dea=18):
    # 计算快的 EMA
    data.loc[:,'EMA_short']=data['close'].ewm(span=short,min_periods=short).mean()
    # 计算慢的 EMA
    data.loc[:,'EMA_long']=data['close'].ewm(span=long,min_periods=long).mean()
    # 计算 MACD 线
    data.loc[:,'DIF']=data['EMA_short']-data['EMA_long']
    # 计算信号线
    data.loc[:,'DEA']=data['DIF'].ewm(span=dea,min_periods=dea).mean()
    # 计算柱状图
    data.loc[:,'MACD_Hist'] = data['DIF']-data['DEA']
    return data

In [5]:
# 实现选股策略找到连续十次下跌
def select_stocks(data):
    # 复制数据框，避免 SettingWithCopyWarning
    data = data.copy()
    # 创建一个布尔列，标记 MACD_Hist 是否比前一天低
    data.loc[:,'Down'] = data['MACD_Hist'].diff() < 0
    
    # 创建一个布尔列，标记连续 十次下跌的情况
    data.loc[:,'Ten_Down'] = data['Down'].rolling(window=11).sum() == 11
    
    # 创建一个标记列，初始化为 0
    data.loc[:,'Selected'] = 0
    
    # 当连续十天下跌时，在第十天下跌的交易日标记为 1
    data.loc[data['Ten_Down'], 'Selected'] = 1
    
    return data

In [6]:
def select_stocks_upward(data):
    # 复制数据框，避免 SettingWithCopyWarning
    data = data.copy()
    
    # 创建一个布尔列，标记 MACD_Hist 是否比前一次高
    data.loc[:, 'Up'] = data['MACD_Hist'].diff() > 0
    
    # 创建一个布尔列，标记连续十次上涨的情况
    data.loc[:, 'Ten_Up'] = data['Up'].rolling(window=10).sum() == 10
    
    # 创建一个标记列，初始化为 0
    data.loc[:, 'Up_Signal'] = 0
    
    # 当连续十次上涨时，在第十次上涨的交易日标记为 2
    data.loc[data['Ten_Up'], 'Up_Signal'] = 2
    
    return data

In [7]:
#  计算 MACD_Hist差值
def down_num(data):
    data['Down'] = data['MACD_Hist'].diff()
    data['Down'] = data['Down'].apply(lambda x: round(x, 4))
    return data

In [8]:
# 检查是否存在 'stock_code' 列
if 'stock_code' not in df.columns:
    print("No Buy Code")
else:
    # 按股票代码分组并对每组数据应用指标计算
    grouped = df.groupby('stock_code')
    result_dfs = []
    for name, group in grouped:
        group = getmacd(group)
        group = select_stocks(group)
        group = down_num(group)
        group = select_stocks_upward(group)
        result_dfs.append(group)

    # 检查是否有数据
    if not result_dfs:
        print("No Buy Code")
    else:
        # 合并处理后的数据
        df = pd.concat(result_dfs)

        # 选择需要的列
        df = df[['Datetime', 'stock_code', 'open', 'low', 'high', 'close', 'MACD_Hist', 'Selected', 'Down', 'Up_Signal', 'volume']]

        # 导出为CSV文件，保留索引并使用新的索引名称
        df.to_csv('C:/Python/Vanyosr_Strategy/Min_Signal/Vanyosr_1m_stocks.csv', index=True, index_label='Index')

No Buy Code


In [9]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('C:/Python/Vanyosr_Strategy/Min_Signal/Vanyosr_1m_stocks.csv')

# 初始化 'Signal' 和 'stop' 列，'stop'用于记录止损点
df['Signal'] = None
df['stop'] = None

# 按股票代码分组处理
grouped = df.groupby('stock_code')

# 函数用于处理每个股票组
def process_group(group):
    buy_signal_flag = False
    buy_price = None
    stop_loss_price = None
    for i in range(1, len(group)):
        if not buy_signal_flag and group.iloc[i-1]['Selected'] == 1 and group.iloc[i]['Selected'] == 0:
            group.loc[group.index[i], 'Signal'] = 'B'  # 当天赋值为 "B"
            buy_signal_flag = True  # 记录买入信号
            buy_price = group.iloc[i]['close']  # 记录买入时的收盘价
            stop_loss_price = buy_price * 0.96  # 初始止损价
        
        if buy_signal_flag:
            current_price = group.iloc[i]['close']
            # 检查是否触发止损
            if current_price <= stop_loss_price:
                group.loc[group.index[i], 'Signal'] = 'K'  # 止损卖出
                buy_signal_flag = False  # 清除买入信号
                buy_price = None  # 清空买入价格
    
    return group

# 应用函数到每个分组
df = grouped.apply(process_group)

# 查看结果
print(df[['open', 'low', 'high', 'close', 'volume', 'MACD_Hist', 'Selected', 'Up_Signal', 'Signal', 'Down', 'stop']])

# 保存结果到新的 CSV文件
df.to_csv('C:/Python/Vanyosr_Strategy/Min_Signal/Signal_1m.csv', index=False)

        open    low   high  close  volume  MACD_Hist  Selected  Up_Signal  \
0      15.39  15.37  15.39  15.38     490        NaN         0          0   
1      15.38  15.37  15.38  15.38     516        NaN         0          0   
2      15.38  15.38  15.40  15.40     761        NaN         0          0   
3      15.39  15.39  15.42  15.41     735        NaN         0          0   
4      15.41  15.39  15.41  15.39     737        NaN         0          0   
...      ...    ...    ...    ...     ...        ...       ...        ...   
10544  21.54  21.53  21.54  21.54     461   0.000285         0          0   
10545  21.54  21.53  21.54  21.54     293   0.000432         0          0   
10546  21.54  21.54  21.54  21.54       1   0.000566         0          0   
10547  21.54  21.54  21.54  21.54       0   0.000687         0          0   
10548  21.54  21.54  21.54  21.54     918   0.000795         0          0   

      Signal    Down  stop  
0       None     NaN  None  
1       None     

C:\Users\33615\AppData\Local\Temp\ipykernel_15996\1711694489.py:36: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = grouped.apply(process_group)


# 移动止盈 移动止损

In [10]:
# import pandas as pd

# # 读取CSV文件
# df = pd.read_csv('C:/Python/Vanyosr Strategy/Min Signal/Vanyosr_1m_stocks.csv')

# # 初始化 'Signal' 和 'stop' 列，'stop'用于记录止损点
# df['Signal'] = None
# df['stop'] = None

# # 按股票代码分组处理
# grouped = df.groupby('stock_code')

# # 函数用于处理每个股票组
# def process_group(group):
#     buy_signal_flag = False
#     buy_price = None
#     stop_loss_price = None
#     target_profit_price = None
#     for i in range(1, len(group)):
#         if not buy_signal_flag and group.iloc[i-1]['Selected'] == 1 and group.iloc[i]['Selected'] == 0:
#             group.loc[group.index[i], 'Signal'] = 'B'  # 当天赋值为 "B"
#             buy_signal_flag = True  # 记录买入信号
#             buy_price = group.iloc[i]['close']  # 记录买入时的收盘价
#             stop_loss_price = buy_price * 0.96  # 初始止损价
#             target_profit_price = buy_price * 1.03  # 初始止盈价
        
#         if buy_signal_flag:
#             current_price = group.iloc[i]['close']
#             # 更新止损和止盈价格
#             if current_price >= target_profit_price:
#                 target_profit_price = current_price * 1.03  # 更新止盈价
#                 stop_loss_price = current_price * 0.99  # 更新止损价

#             # 检查是否触发止损或止盈
#             if current_price <= stop_loss_price:
#                 group.loc[group.index[i], 'Signal'] = 'K'  # 止损卖出
#                 buy_signal_flag = False  # 清除买入信号
#                 buy_price = None  # 清空买入价格
#             elif current_price >= target_profit_price:
#                 group.loc[group.index[i], 'Signal'] = 'T'  # 止盈卖出
#                 buy_signal_flag = False  # 清除买入信号
#                 buy_price = None  # 清空买入价格
        
#     return group

# # 应用函数到每个分组
# df = grouped.apply(process_group)

# # 查看结果
# print(df[['open', 'low', 'high', 'close', 'volume', 'MACD_Hist', 'Selected', 'Up_Signal', 'Signal', 'Down', 'stop']])

# # 保存结果到新的 CSV文件
# df.to_csv('C:/Python/Vanyosr Strategy/Min Signal/Signal_1m.csv', index=False)

In [11]:
import pandas as pd
import math

# 读取分时数据文件
df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Min_Signal\\Signal_1m.csv')

# 初始化账户状态
initial_cash = 100000  # 初始资金
available_cash = initial_cash
portfolio = {}  # 股票持仓
trade_log = []  # 记录交易
total_cash_log = []  # 总资金列表

# 获取所有日期按顺序排列
dates = sorted(df['Datetime'].unique())

# 获取所有独特的股票代码
unique_stocks = df['stock_code'].unique()
stock_count = len(unique_stocks)  # 股票数量

# 设置每只股票最小买入股数
MIN_SHARES = 100  # 每次买入最少100股

# 假设的滑点和手续费率
slippage = 0.0002  # 0.02%的滑点
buy_commission_rate = 0.0001  # 买入手续费率 0.01%
sell_commission_rate = 0.0002  # 卖出手续费率 0.02%

# 交易逻辑
for date in dates:
    # 筛选出当天有买入信号的股票
    signals_today = df[(df['Datetime'] == date) & (df['Signal'] == 'B')].copy()

    # 计算每只股票的资金分配
    if stock_count > 0:
        position_size = available_cash / stock_count

    for idx, row in signals_today.iterrows():
        stock_code = row['stock_code']
        buy_price = row['close']
        
        # 计算滑点后的买入价格
        adjusted_buy_price = buy_price * (1 + slippage)
        
        shares_to_buy = math.floor(position_size / adjusted_buy_price / MIN_SHARES) * MIN_SHARES
        total_cost = shares_to_buy * adjusted_buy_price * (1 + buy_commission_rate)
        
        if shares_to_buy >= MIN_SHARES and total_cost <= position_size:
            portfolio[stock_code] = {
                'buy_price': adjusted_buy_price,
                'quantity': shares_to_buy,
                'buy_date': date
            }
            available_cash -= total_cost
            
            trade_log.append({
                'date': date,
                'stock': stock_code,
                'action': 'buy',
                'price': adjusted_buy_price,
                'quantity': shares_to_buy,
                'cash_remaining': available_cash
            })

    # 卖出逻辑
    for stock in list(portfolio.keys()):
        stock_data = df[(df['stock_code'] == stock) & (df['Datetime'] == date)]
        if not stock_data.empty and stock_data.iloc[0]['Signal'] in ['S', 'T', 'K']:
            sell_price = stock_data.iloc[0]['close']
            adjusted_sell_price = sell_price * (1 - slippage)
            quantity = portfolio[stock]['quantity']
            total_sell_value = adjusted_sell_price * quantity * (1 - sell_commission_rate)
            available_cash += total_sell_value
            
            trade_log.append({
                'date': date,
                'stock': stock,
                'action': 'sell',
                'price': adjusted_sell_price,
                'quantity': quantity,
                'cash_remaining': available_cash
            })
            
            del portfolio[stock]

    # 更新所有持仓股票的市值
    total_stock_value = 0
    for stock, details in portfolio.items():
        current_price = df[(df['stock_code'] == stock) & (df['Datetime'] == date)]['close'].iloc[-1]
        total_stock_value += details['quantity'] * current_price
    
    total_cash = available_cash + total_stock_value
    
    total_cash_log.append({
        'date': date,
        'total_cash': total_cash
    })

# 打印交易记录
trade_log_df = pd.DataFrame(trade_log)
print(trade_log_df)

# 打印总资金记录
total_cash_df = pd.DataFrame(total_cash_log)
print(total_cash_df)

# 保存交易记录和总资金记录到 CSV文件
trade_log_df.to_csv('C:\\Python\\Vanyosr_Strategy\\Min_Signal\\trade_log_1m.csv', index=False)
total_cash_df.to_csv('C:\\Python\\Vanyosr_Strategy\\Min_Signal\\total_cash_log_1m.csv', index=False)

下载进度： {'finished': 2, 'total': 8, 'stockcode': '', 'message': '600938.SH'}
下载进度： {'finished': 3, 'total': 8, 'stockcode': '', 'message': '601919.SH'}
下载进度： {'finished': 4, 'total': 8, 'stockcode': '', 'message': '601919.SH'}
下载进度： {'finished': 5, 'total': 8, 'stockcode': '', 'message': '600426.SH'}
下载进度： {'finished': 6, 'total': 8, 'stockcode': '', 'message': '600426.SH'}
下载进度： {'finished': 7, 'total': 8, 'stockcode': '', 'message': '000807.SZ'}
下载进度： {'finished': 8, 'total': 8, 'stockcode': '', 'message': '000807.SZ'}
             date      stock action      price  quantity  cash_remaining
0  20250528101000  601156.SH    buy  13.122624      1000    86876.063738
1  20250620100900  601021.SH    buy  55.071012       200    75860.759917
2  20250624095900  000807.SZ    buy  15.703140       600    66437.933729
3  20250624100200  603613.SH    buy  21.514302       400    57831.352357
4  20250625103200  600426.SH    buy  21.634326       300    51340.405527
5  20250626102500  600938.SH    buy  

In [12]:
import pandas as pd

# 读取交易日志文件
trade_log_df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Min_Signal\\trade_log_1m.csv')

# 读取分时数据文件以获取最终的收盘价
df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Min_Signal\\Signal_1m.csv')

# 函数计算每只股票的总收益百分比
def calculate_profit_percentage(trade_log, final_prices):
    profits_percentage = {}
    for stock in trade_log['stock'].unique():
        stock_trades = trade_log[trade_log['stock'] == stock]
        total_bought = 0
        total_sold = 0
        last_buy_price = 0

        for index, row in stock_trades.iterrows():
            if row['action'] == 'buy':
                total_bought += row['price'] * row['quantity']
                last_buy_price = row['price']  # 更新最后的买入价格
            elif row['action'] in ['sell', 'T', 'K']:
                total_sold += row['price'] * row['quantity']

        # 如果最后一笔交易是买入，使用最终价格计算未卖出部分的价值
        if stock_trades.iloc[-1]['action'] == 'buy':
            final_price = final_prices[final_prices['stock_code'] == stock]['close'].iloc[-1]
            total_sold += final_price * stock_trades.iloc[-1]['quantity']

        if total_bought > 0:  # 避免除以零
            profit_percentage = (total_sold - total_bought) / total_bought * 100
            profits_percentage[stock] = round(profit_percentage, 2)

    return profits_percentage

# 函数计算每只股票的交易笔数
def count_trade_transactions(trade_log):
    trade_transactions = {}
    for stock in trade_log['stock'].unique():
        stock_trades = trade_log[trade_log['stock'] == stock]
        # 买入和卖出成对出现，因此计算买入次数即可
        buy_count = len(stock_trades[stock_trades['action'] == 'buy'])
        trade_transactions[stock] = buy_count
    return trade_transactions

# 函数计算涨跌幅
def calculate_price_change(trade_log, final_prices):
    price_changes = {}
    for stock in trade_log['stock'].unique():
        initial_price = trade_log[(trade_log['stock'] == stock) & (trade_log['action'] == 'buy')]['price'].iloc[0]
        final_price = final_prices[final_prices['stock_code'] == stock]['close'].iloc[-1]
        price_change = (final_price - initial_price) / initial_price * 100  # 百分比涨跌幅
        price_changes[stock] = round(price_change, 2)  # 保留两位小数
    return price_changes

# 函数计算盈利次数
def calculate_profitable_trades(trade_log):
    profit_counts = {}
    for stock in trade_log['stock'].unique():
        stock_trades = trade_log[trade_log['stock'] == stock]
        buy_prices = stock_trades[stock_trades['action'] == 'buy']['price'].tolist()
        sell_prices = stock_trades[stock_trades['action'] == 'sell']['price'].tolist()
        profits = sum(1 for buy, sell in zip(buy_prices, sell_prices) if sell > buy)
        profit_counts[stock] = profits
    return profit_counts

# 计算统计数据
profits_percentage = calculate_profit_percentage(trade_log_df, df)
trade_transactions = count_trade_transactions(trade_log_df)
price_changes = calculate_price_change(trade_log_df, df)
profitable_trades = calculate_profitable_trades(trade_log_df)

# 创建DataFrame来展示结果
results_df = pd.DataFrame({
    'Stock': list(profits_percentage.keys()),
    'Trade Percentage(%)': list(profits_percentage.values()),
    'Trade Count': [trade_transactions.get(stock, 0) for stock in profits_percentage.keys()],
    'Win Trades': [profitable_trades.get(stock, 0) for stock in profits_percentage.keys()],
    'Price Change(%)': [price_changes.get(stock, 0) for stock in profits_percentage.keys()]
})

# 打印结果
print(results_df)

       Stock  Trade Percentage(%)  Trade Count  Win Trades  Price Change(%)
0  601156.SH                 0.28            1           0             0.28
1  601021.SH                 1.38            1           0             1.38
2  000807.SZ                 3.10            1           0             3.10
3  603613.SH                 0.12            1           0             0.12
4  600426.SH                 0.35            1           0             0.35
5  600938.SH                -1.01            1           0            -1.01
